# REPORT: Predicting Risk of Heart Disease from Accessible Health Metrics
_Created by : Tyler Ofreneo, Arthur Zhang, Kody Forsyth, Natalia Blanco_

## Introduction:

According to the Public Health Agency of Canada, heart disease is the second leading cause of death in Canada, with approximately 1 in 12 Canadian adults over 20 living with a diagnosis. These metrics highlight the importance of knowing the risk factors and having access to medical advice. However, a shortage of physicians in Canada is causing a lack of available health care (Flood et al., 2023). Non-healthcare professionals do not have the means to properly self-evaluate symptoms, therefore our project seeks to help the general population to make informative decisions about heart disease symptoms that are self-monitored or easily accessible.


Thus we ask, <b>is it possible to classify individuals into levels of heart disease risk (low risk, moderate risk, or high risk) based on blood pressure, cholesterol, heart rate and chest pain?<b>


Our analysis will use the Heart Disease dataset from the Cleveland database for heart disease (Andras et al., 1988). This database consists of 303 patients without history of heart disease, who were admitted to the Cleveland Clinic between 1981 and 1984. 



In [ ]:
# Please uncomment the following cell to install the altair in case your package is not up-to-date

In [ ]:
pip install -U altair

In [ ]:
import altair as alt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.compose import make_column_transformer
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline

## Loading and Wrangling the Heart Disease Data Set

The Heart Disease dataset retrieved from the UC Irvine Machine Learning Repository is loaded into this notebook in the following cell. The data frame obtained (shown below) was tidy (Table 1). A subset of the columns containing the symptoms chosen in the thesis was extracted. To enhance clarity, column names and variables within the 'diagnosis' and 'chest pain' sections have been given more meaningful names. An outline of the the classes for heart disease diagnosis, each representing the levels of risk, is given below.

> <ins> The 'diagnosis' column: </ins> 
<br>
> 0,1 is represented as "low-risk diagnosis"
<br>
> 2,3 is represented as "moderate-risk diagnosis"
<br>
> 4 is represented as "high-risk diagnosis"

In [ ]:
# import and load the data
heart_disease = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")

#extract and rename the columns with required data
heart_disease = heart_disease[["chol","cp","thalach","trestbps","num"]]
heart_disease.rename(columns = {
                          "chol" : "cholesterol", 
                          "cp":"type_chestpain",
                          "thalach" : "max_heart_rate",
                          "num" : "diagnosis",
                          "trestbps" : "resting_bp"
}, inplace = True)

# renaming variables in the 'diagnosis' column with readable and explicit labels
heart_disease['diagnosis'] = heart_disease['diagnosis'].replace([0,1], "low-risk heart disease")
heart_disease['diagnosis'] = heart_disease['diagnosis'].replace([2,3], "moderate-risk heart disease")
heart_disease['diagnosis'] = heart_disease['diagnosis'].replace([4], "high-risk heart disease")

# renaming variables in the 'chest pain' column with readable and explicit labels
heart_disease['type_chestpain'] = heart_disease['type_chestpain'].replace(
    [1,2,3,4],
    ["type1","type2","type3","type4"])

print("Table 1:")
heart_disease.head()

## Balancing

As shown in the next cells, the unique values in the 'diagnosis' column show a class imbalance because the observations of 'low-risk heart disease' appear more common than the other classes. In order to eliminate the risk of bias in the classifier, this issue was resolved by oversampling the rare classes which were "moderate-" and "high-risk disease". This was done by filtering the classes and randomly resampling to increase the number of rare observations.

In [ ]:
# unbalanced diagnoses, must resample
heart_disease['diagnosis'].value_counts()

In [ ]:
# separate the classes out into their own data frames by filtering
rare_diagnosis_1 = heart_disease[heart_disease["diagnosis"] == "high-risk heart disease"]
rare_diagnosis_2 = heart_disease[heart_disease["diagnosis"] == "moderate-risk heart disease"]
low_risk_diagnosis = heart_disease[heart_disease["diagnosis"] == "low-risk heart disease"]

# increase the number of rare observations to the same amount as non-rare observations
rare_diagnosis_upsample_1 = resample(rare_diagnosis_1, n_samples = low_risk_diagnosis.shape[0])
rare_diagnosis_upsample_2 = resample(rare_diagnosis_2, n_samples = low_risk_diagnosis.shape[0])

# combined observations show that the classes are now balanced
heart_disease = pd.concat((rare_diagnosis_upsample_1, rare_diagnosis_upsample_2, low_risk_diagnosis)).reset_index(drop = True)
heart_disease['diagnosis'].value_counts()

## Create Training and Testing Sets

The data set is split so that 75% is incorporated into a training set and 25% for a testing set. The data is stratified by the categorical label variable ('diagnosis'). We will use the training set to build the classifier and predict the diagnosis of observations in the test set for evaluating performance. 

In [ ]:
# split data into training and testing sets
heart_disease_train, heart_disease_test = train_test_split(heart_disease, 
                                                           train_size = 0.75, 
                                                           stratify = heart_disease["diagnosis"],
                                                           random_state = 0)

## Summary Statistics

The following cells are used to compute the summary statistics of the variables.
<br>
<br>
**Categorical variables (Table 2)**: 
> Number of observations ("count")<br>
> Number of unique labels ("unique")<br>
> Most frequent value ("top") <br>
> Regularity of the top value ("freq")<br>

**Continuous variables (Table 3)**: 
> Number of observations ("count")<br>
> Mean, standard deviation, minimum and maximum of the observations ("mean", "std", "min", "max")<br>
> 25th, 50th, and 75th percentiles<br>

In [ ]:
# Summary of the categorical variables
heart_disease_categorical = heart_disease_train.drop(columns = ["cholesterol","max_heart_rate","resting_bp"])
print("Table 2:")
heart_disease_categorical.describe()

In [ ]:
# Summary of the continuous variables
heart_disease_continuous = heart_disease_train.drop(columns = ["type_chestpain","diagnosis"])
print("Table 3:")
heart_disease_continuous.describe()

## Distribution of the Variables

The next cells visualize the distribution of the measurements/values for each symptom variable in histograms (Figure 1-4). The associated heart disease risk diagnosis is represented with distinct colours. The varying measurements indicate standardization is required. 

In [ ]:
# distribution of blood pressure measurements
bp_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("resting_bp:Q", bin = True).title("Blood Pressure"),
    y=alt.Y("count()").stack(False),
    color="diagnosis:N"
).properties(
    title = "Figure 1: Distribution of Blood Pressure"
)

# distribution of cholesterol measurements
chol_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("cholesterol:Q", bin = True).title("Cholesterol"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    title = "Figure 2: Distribution of Cholesterol"
)

# distribution of chest pain type measurements
cp_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("type_chestpain").title("Chest Pain Type"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    width=300,
    height=300,
    title = "Figure 3: Distribution of Chest Pain Type"
)

# distribution of heart rate measurements
hr_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("max_heart_rate:Q", bin = True).title("Heart Rate"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    title = "Figure 4: Distribution of Heart Rate"
)


combined_charts = bp_hist & cp_hist | chol_hist & hr_hist
combined_charts

## Note: "Chest Pain" Variable

From here, our team decided that in order to use the chest pain symptoms variable effectively (column: "type_chestpain", Figure 3), a larger and more comprehensive analysis is needed, potentially involving the creation of separate classifiers for each label of chest pain. The nature of this analysis exceeds the scope of this project thus it was not pursued. However, the work above has included chest pain for future analysis that builds upon this classification. 

## Finding the K Parameter 

A pipeline is created that combines the preprocessing and an undefined K-neighbors classifier. The parameter values are collected within a range and utilized to estimate k-classifier accuracy. The classifiers are tuned according to the symptoms and diagnosis labels and the output of the cross-validation accuracy estimate for each K parameter is visualized with a line plot (Figure 5).

In [ ]:
# create a preprocessor
preprocessor=make_column_transformer(
    (StandardScaler(),['cholesterol','max_heart_rate','resting_bp']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

# create a pipeline
heart_disease_pipe = make_pipeline(preprocessor, KNeighborsClassifier())
heart_disease_pipe

In [ ]:
np.random.seed(1234)

# create a parameter grid
parameter_grid = {
    "kneighborsclassifier__n_neighbors" : range(1, 31)
}

# estimate the classifier accuracy for the range of values
grid_search = GridSearchCV(
    estimator = heart_disease_pipe,
    param_grid = parameter_grid,
    cv = 5,
)

X_heart_train=heart_disease_train[['cholesterol','max_heart_rate','resting_bp']]
y_heart_train=heart_disease_train['diagnosis']

# tuning process
model_grid=grid_search.fit(X_heart_train,y_heart_train)
grid_results=pd.DataFrame(grid_search.cv_results_)

In [ ]:
# plot the accuracy of the K-values with a line-plot
cross_val_plot = alt.Chart(grid_results).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("Values for K").scale(zero=True),
    y=alt.Y("mean_test_score").title("Accuracy of model").scale(zero=False)
).properties(
    title = "Figure 5: Estimated Accuracy of K Neighbors"
)

cross_val_plot

## Finding the K Parameter: Result

The best K Neighbor value is 1. This value provides optimal accuracy (Figure 5), it is also not too large to become prohibitive of the model and changing the value to a nearby number would not change the accuracy to a great extent. The following code creates a new classifier with this parameter and the predictions from using this model on the test data are added to the test set. Some of the predicted ('predicted') and actual ('diagnosis') diagnoses are shown (Table 4). 

In [ ]:
# create a new model object for the best parameter value
knn = KNeighborsClassifier(n_neighbors = 1)
heart_fit = knn.fit(X_heart_train,y_heart_train)

# add predictions to dataframe
heart_predictions = heart_disease_test.assign(predicted = heart_fit.predict(heart_disease_test[['cholesterol', 'max_heart_rate', 'resting_bp']]))
print('Table 4:')
heart_predictions.head()

## Evaluation Results

The next cells show that the model has high accuracy, at 93%.

A confusion matrix displays all observations (54) for high-risk heart disease, 39 observations for low-risk heart disease and 56 observations of moderate-risk heart disease were correctly predicted. The classifier made minimal misdiagnosis in predictions of low- and moderate-risk.

Based on the confusion matrix, here are the calculations for precision and recall:
<br>
<br>
_high-risk heart disease_

>precision = 54/54 100 = 100.0 = 100% <br>
>recall = 54/56 100 = 96.42 = 96%

_low-risk heart disease_

>precision = 39/51 100 = 76.47 = 76% <br>
>recall = 39/43 100 = 90.70 = 91%

_moderate-risk heart disease_

>precision = 56/60 100 = 93.33 = 93% <br>
>recall = 56/66 100 = 84.85 = 85%

In [ ]:
# test model's accuracy 
heart_disease_correct = heart_predictions[
    heart_predictions['diagnosis'] == heart_predictions['predicted']
] 
heart_disease_acc = heart_disease_correct.shape[0] / heart_predictions.shape[0]
heart_disease_acc

In [ ]:
# confusion matrix
confusion_matrix = pd.crosstab(
    heart_predictions['diagnosis'],
    heart_predictions['predicted'],
    rownames=['Actual'],
    colnames=['Predicted']
)

print('Table 5:')
confusion_matrix

## Visualization of Analysis

Here, the visualization depicts the results of this classification analysis: the predictions made by the classifier for each label of heart disease risk diagnosis are described in bar plots to easily compare the results. The first plot (Figure 6) shows the distributions of predictions correct and incorrect predicted instances and visualizes recall. The second plot (Figure 7) captures the amount of diagnosis and the associated amount of correct prediction, visualizing precision.

In [ ]:
predictions_hist = alt.Chart(heart_predictions).mark_bar().encode(
    x = alt.X('predicted').title('Predicted Diagnosis'),
    y = alt.Y('count()').title('Count of Predicted Diagnosis'),
    column = alt.Column('diagnosis').title('Figure 6: Diagnosis'),
    color = alt.Color('predicted').title('Predicted Diagnosis'),
).properties(
    height = 350,
    width = 350,
).configure_axisX(labelAngle = -40, titleFontSize = 12).configure_axis(titleFontSize = 12, labelFontSize = 12, labelAlign = 'center', labelPadding = 40)

predictions_hist

In [ ]:
# Isolate the predicted and actual diagnosis
heart_predictions = heart_predictions[['diagnosis', 'predicted']]

# create a dataframe for the bar chart
heart_disease_result = pd.DataFrame({
    'type': ['diagnosis',
             'predicted',
             'diagnosis',
             'predicted',
             'diagnosis',
             'predicted'],
    'risk': ['low-risk heart disease',
             'low-risk heart disease',
             'moderate-risk heart disease',
             'moderate-risk heart disease',
             'high-risk heart disease',
             'high-risk heart disease'],
    'count' : [heart_predictions[(heart_predictions['diagnosis'] == 'low-risk heart disease')].shape[0],
               heart_predictions[(heart_predictions['predicted'] == 'low-risk heart disease')].shape[0],
               heart_predictions[(heart_predictions['diagnosis'] == 'moderate-risk heart disease')].shape[0],
               heart_predictions[(heart_predictions['predicted'] == 'moderate-risk heart disease')].shape[0],
               heart_predictions[(heart_predictions['diagnosis'] == 'high-risk heart disease')].shape[0],
               heart_predictions[(heart_predictions['predicted'] == 'high-risk heart disease')].shape[0]]
})

# draw the bar chart
heart_disease_result_bar = alt.Chart(heart_disease_result).mark_bar().encode(
    x = alt.X('type', title = 'Diagnosis Type'),
    y = alt.Y('count', title = 'Count of Diagnosis'),
    color = alt.Color('risk', title = 'Diagnosis Type'),
    column = alt.Column('risk', title = 'Figure 7: Risk Type')
).properties(
    height = 350,
    width = 350,
).configure_axisX(labelAngle = 0, titleFontSize = 12).configure_axis(titleFontSize = 12, labelFontSize = 12, labelAlign = 'center', labelPadding = 40)

heart_disease_result_bar


## Discussion

This classification analysis suggested that there is a link between the specific health metrics of blood pressure, cholesterol and heart rate to the diagnosis of heart disease. An evaluation of the model created in this analysis showed a high accuracy which provides confidence in the predictions of the classifier. High levels of precision and recall for each class of heart disease diagnosis indicate robust performance in correctly identifying diagnosis. Notably, the highest calculated precision and recall were provided for the high-risk heart disease class, which is highly beneficial for the utility of this model due to the severity of this particular diagnosis. 

Based on several previous research studies, our team expected a strong relationship between heart disease diagnosis and blood pressure, cholesterol and heart rate symptoms (Fuchs et al., 2020; Hjalmarson et al., 2007; Arnold et al., 2008). Our expectations aligned well with the effectiveness demonstrated by the K Neighbors Classification model in predicting diagnoses based on these symptoms.

The results of this project may help inform the general population of their susceptibility to heart disease based on self-monitored health metrics or symptoms. By designing a model for recognizing heart disease risk using common symptoms, we have provided context for the severity of symptoms that require urgent health care consultations. Furthermore, this project will allow future studies to question whether genetic, environmental or demographics can refine these predictions.

## References

Wiliamson, Laura. “Undiagnosed Heart Disease May Be Common in People with Heart Attacks Not Caused by Clots.” Www.Heart.Org, American Heart Association News, 24 Jan. 2023, www.heart.org/en/news/2022/03/28/undiagnosed-heart-disease-may-be-common-in-people-with-heart-attacks-not-caused-by-clots

Fuchs FD, Whelton PK. High Blood Pressure and Cardiovascular Disease. Hypertension. 2020 Feb;75(2):285-292. doi: 10.1161/HYPERTENSIONAHA.119.14240. Epub 2019 Dec 23. PMID: 31865786; PMCID: PMC10243231.

Åke Hjalmarson, Heart rate: an independent risk factor in cardiovascular disease, European Heart Journal Supplements, Volume 9, Issue suppl_F, September 2007, Pages F3–F7, https://doi.org/10.1093/eurheartj/sum030

Arnold JM, Fitchett DH, Howlett JG, Lonn EM, Tardif JC. Resting heart rate: a modifiable prognostic indicator of cardiovascular risk and outcomes? Can J Cardiol. 2008 May;24 Suppl A(Suppl A):3A-8A. doi: 10.1016/s0828-282x(08)71019-5. PMID: 18437251; PMCID: PMC2787005.

Haasenritter J, Stanze D, Widera G, Wilimzig C, Abu Hani M, Sonnichsen AC, Bosner S, Rochon J, Donner-Banzhoff N. Does the patient with chest pain have a coronary heart disease? Diagnostic value of single symptoms and signs--a meta-analysis. Croat Med J. 2012 Oct;53(5):432-41. doi: 10.3325/cmj.2012.53.432. PMID: 23100205; PMCID: PMC3490454.

Janosi,Andras, Steinbrunn,William, Pfisterer,Matthias, and Detrano,Robert. (1988). Heart Disease. UCI Machine Learning Repository. https://doi.org/10.24432/C52P4X.